### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [1]:
%%time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, \
                        accuracy_score, classification_report, roc_auc_score, roc_curve

CPU times: user 469 ms, sys: 181 ms, total: 650 ms
Wall time: 935 ms


## Шаг 1. Импортируем все в Pandas


In [2]:
%%time
sample_submission = pd.read_csv('sampleSubmission.csv',',')
train_df = pd.read_csv('train.csv','\t')
test_df = pd.read_csv('test.csv','\t')
other_df = pd.read_csv('other.csv','\t')

CPU times: user 17.6 s, sys: 2.02 s, total: 19.6 s
Wall time: 20 s


In [4]:
train_df['concat'] = train_df['name']+' '+ train_df['description']
train_df.head()

,id,name,description,target,concat
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,Заведующий отделом/секцией в магазин YORK (Уру...
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Наладчик станков и манипуляторов с ПУ Обязанно...
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Разработчик С++ (Криптограф) <strong>Требовани...
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Мерчендайзер/продавец-консультант <p><strong>К...


## Шаг 2. Чистим данные и подготавливаем текст

In [5]:
# Подключаем БьютифулСуп
from bs4 import BeautifulSoup             

# Прогоняем отдельное описание через БьютифулСуп   
example1 = BeautifulSoup(train_df['concat'][0], 'html5lib')  

In [6]:
#Подключаем регулярки, чтобы очистить текст от тегов html
import re
# Делаем найти и заменить на ничто для всего что не похоже на регулярку
letters_only = re.sub("[^а-яА-Яa-zA-Z]",           # находим всё что не
                      " ",                   # Меняем на пустоту
                      example1.get_text() )  
print (letters_only)

Заведующий отделом секцией в магазин YORK  Уручье  В НОВЫЙ МАГАЗИН YORK  хозтовары  приглашаем на постоянную работу руководителя секции  хозтовары   Обязанности      организация эффективного и культурного обслуживания покупателей организация приемки сдачи товаров на склад консультация покупателей по вопросам  касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества  сроков годности ТМЦ  наличие маркировок  ценников на товарах организация бесперебойной работы товарной секции и участка склада  закрепленного за ней контроль за сохранностью товаров  торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования   знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца   р    заместителя заведующего секцией отделом ответственность  лидерские качества  опыт управления коллективом      Условия   НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив д

In [7]:
lower_case = letters_only.lower()
words = lower_case.split()  

In [8]:
import nltk

#nltk.download()  # Если NLTK не установлен, то надо раскомментровать этот блок и исполнить

In [9]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
print( stopwords.words('russian') )

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [10]:
# Удаляем стоп-слова русского языка из текста words
words = [w for w in words if not w in stopwords.words('russian')]
print (words)
# Посмоторим что получится, если прогнать стеммером. 
# Но вариант со стеммером будет в следующей версии домашней работы
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer( 'russian' )
words = [snowball_stemmer.stem(w) for w in words if not w in stopwords.words('russian')]
print (words)

['заведующий', 'отделом', 'секцией', 'магазин', 'york', 'уручье', 'новый', 'магазин', 'york', 'хозтовары', 'приглашаем', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары', 'обязанности', 'организация', 'эффективного', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'склад', 'консультация', 'покупателей', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'участка', 'склада', 'закрепленного', 'контроль', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'розничной', 'торговле', 'качестве', 'продавца', 'р', 'заместителя', 'за

In [11]:
def vac_to_words( description, Stemmer ):
    """
    Функция делает преобразование поля description из формата html в формат с отдельными значащими словами
    На вход передается  описание и параметр True/False. Если True, то работает стеммер, иначе нет
    """
    description_text = BeautifulSoup(description, 'html5lib').get_text() #получаем текст      
    letters_only = re.sub('[^а-яА-Яa-zA-Z]', ' ', description_text) #удаляем всё что не буквы
    words = letters_only.lower().split() #получаем слова в нижнем регистре                             
    stops = set(stopwords.words('russian'))  # формируем set стоп слов русского языка                  
    if Stemmer: #удаляем минус фразы и  оставляем либо "стеммы", либо значащие слова
        meaningful_words = [snowball_stemmer.stem(w) for w in words if not w in stops]
    else: 
        meaningful_words = [w for w in words if not w in stops]
    return( ' '.join( meaningful_words ))  # собираем слова в строку с разделителеим пробел 

In [12]:
clean_vac = vac_to_words( train_df['concat'][0] , True)
print(clean_vac)

завед отдел секц магазин york уруч нов магазин york хозтовар приглаша постоя работ руководител секц хозтовар обязан организац эффективн культурн обслуживан покупател организац приемк сдач товар склад консультац покупател вопрос каса оказыва услуг контрол своевремен подач товар торгов секц проверк качеств срок годност тмц налич маркировок ценник товар организац бесперебойн работ товарн секц участк склад закреплен контрол сохран товар торгов оборудован проч материальн ценност инвентаризац товар требован знан товарн групп знан прав приемк товар оп работ розничн торговл качеств продавц р заместител завед секц отдел ответствен лидерск качеств оп управлен коллектив услов нов современ магазин розничн торговл хозтовар хорош коллект достойн зарплат прем соц пакет график работ пн пт плава мест работ ул никифоров уруч


In [13]:
# Посмотрим сколько всего описаний есть в train_df
num_vacs = train_df['concat'].size
num_vacs

200000

In [14]:
%%time
clean_train_vacs = [] #делаем пустой лист для записи всех описаний с отдельными словами

print ('Очищаем и собираем нормализованные описания из train_df...\n')

# Пройдем в цикле по всем описаниям
for i in range( 0, num_vacs ):
    
    # Для каждого описания вызываем функцию нормировки текста описания и добавляем нормированный текст в лист
    
    if( (i+1)%10000 == 0 ): #чтобы не было скучно, каждые 10 000 описаний будем печатать развлекательный текст
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_vacs,(i+1)/num_vacs )) 
        
    clean_train_vacs.append( vac_to_words( train_df['concat'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из train_df...

Описание 10000 из 200000 в обработке... 5%
Описание 20000 из 200000 в обработке... 10%
Описание 30000 из 200000 в обработке... 15%
Описание 40000 из 200000 в обработке... 20%
Описание 50000 из 200000 в обработке... 25%
Описание 60000 из 200000 в обработке... 30%
Описание 70000 из 200000 в обработке... 35%
Описание 80000 из 200000 в обработке... 40%
Описание 90000 из 200000 в обработке... 45%
Описание 100000 из 200000 в обработке... 50%
Описание 110000 из 200000 в обработке... 55%
Описание 120000 из 200000 в обработке... 60%
Описание 130000 из 200000 в обработке... 65%
Описание 140000 из 200000 в обработке... 70%
Описание 150000 из 200000 в обработке... 75%
Описание 160000 из 200000 в обработке... 80%
Описание 170000 из 200000 в обработке... 85%
Описание 180000 из 200000 в обработке... 90%
Описание 190000 из 200000 в обработке... 95%
Описание 200000 из 200000 в обработке... 100%
Я закончила!!!
CPU times: user 29min 23s, sys: 16

## Создаём свойства через Bag of Words

In [15]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 10000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs)

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 17.1 s, sys: 5.26 s, total: 22.4 s
Wall time: 23.1 s


In [16]:
train_data_features.shape

(200000, 10000)

In [17]:
# Посмотрим на получившийся словарь 
vocab = vectorizer.get_feature_names()
print (vocab)

['ab', 'abap', 'abb', 'abby', 'abc', 'abilit', 'abilities', 'abl', 'about', 'abov', 'abroad', 'acceptanc', 'access', 'accordanc', 'according', 'account', 'accountabl', 'accounting', 'accounts', 'accurac', 'accurat', 'achiev', 'achieving', 'acool', 'acquisition', 'acrobat', 'acronis', 'across', 'act', 'action', 'actions', 'activ', 'activel', 'activemq', 'activit', 'activities', 'acum', 'ad', 'adapt', 'add', 'addition', 'additiona', 'address', 'adidas', 'administration', 'administrator', 'adob', 'ads', 'advanc', 'advanced', 'advantag', 'advertising', 'advisor', 'adwords', 'affiliat', 'after', 'ag', 'again', 'against', 'agenc', 'agencies', 'agent', 'agil', 'agreed', 'air', 'aix', 'ajax', 'akk', 'al', 'algorithms', 'align', 'all', 'allow', 'allowanc', 'allows', 'along', 'alp', 'alread', 'als', 'altium', 'always', 'am', 'amazing', 'amazon', 'ambitious', 'americ', 'america', 'amocrm', 'among', 'amount', 'an', 'analysis', 'analyst', 'analysts', 'analytic', 'analytica', 'analytics', 'analyz', 

In [18]:
import numpy as np

# Просуммируем счетчики
dist = np.sum(train_data_features, axis=0)

# Посмотрим сколько раз каждое слово встречается в train_df

for tag, count in zip(vocab, dist):
    print (count, tag)

180 ab
530 abap
166 abb
172 abby
106 abc
3514 abilit
191 abilities
1024 abl
1732 about
233 abov
136 abroad
144 acceptanc
513 access
232 accordanc
654 according
1314 account
121 accountabl
133 accounting
322 accounts
87 accurac
136 accurat
374 achiev
115 achieving
142 acool
157 acquisition
117 acrobat
208 acronis
917 across
194 act
194 action
209 actions
1109 activ
204 activel
82 activemq
261 activit
1420 activities
79 acum
1475 ad
152 adapt
83 add
97 addition
428 additiona
178 address
129 adidas
608 administration
288 administrator
2436 adob
120 ads
257 advanc
992 advanced
698 advantag
332 advertising
114 advisor
1852 adwords
197 affiliat
342 after
230 ag
183 again
118 against
215 agenc
184 agencies
93 agent
2812 agil
138 agreed
88 air
79 aix
1249 ajax
141 akk
143 al
299 algorithms
88 align
3032 all
79 allow
104 allowanc
109 allows
116 along
79 alp
117 alread
784 als
240 altium
316 always
267 am
141 amazing
528 amazon
205 ambitious
105 americ
167 america
306 amocrm
416 among
210 amount

163 integrat
143 integrated
1753 integration
113 integrator
134 integrit
382 intel
360 intelligenc
83 intelligent
89 intellij
177 interact
209 interaction
115 interactiv
250 interest
231 interested
678 interesting
120 interests
262 interfac
181 interfaces
5401 intermediat
998 interna
79 internall
2202 internationa
1948 internet
296 interpersona
270 interview
197 interviews
97 intimiss
102 inv
108 inventiv
277 inventor
151 investment
188 inviting
285 involved
193 involving
131 ioc
3186 ios
659 iot
4318 ip
189 ipad
420 iphon
110 ips
159 ipsec
96 iptables
118 iptv
83 ipv
143 irc
13537 is
233 issu
1054 issues
14708 it
154 itechart
92 iterativ
240 itm
851 its
274 itsm
279 iv
218 japa
110 jasmin
8571 jav
7457 javascript
135 jax
221 jboss
284 jdbc
82 je
183 jeans
1027 jenkins
2265 jir
153 jmeter
243 jms
126 jo
1407 job
351 jobs
1868 join
84 joining
390 jooml
104 journe
270 jpa
3057 jquer
5681 js
96 jsf
1071 json
195 jsp
120 jti
301 jun
1313 junior
179 juniper
453 just
111 jvm
112 ka
311 kafk


185 sites
95 six
82 siz
320 sketch
172 skill
261 skilled
8065 skills
476 sku
126 skyeng
945 skyp
340 sla
205 slack
339 small
840 smart
4776 smm
297 sms
179 smtp
150 snacks
104 snmp
987 so
1040 soap
95 soapu
1428 socia
84 societ
81 socket
341 soft
6333 softwar
86 solaris
1391 solid
689 solidworks
85 solr
1278 solution
3501 solutions
378 solv
81 solver
814 solving
585 som
123 someon
94 something
197 son
110 sound
769 sourc
144 sources
439 spa
398 spac
92 spacious
276 spain
94 spar
341 spark
88 spc
129 speak
91 speaker
97 speakers
284 speaking
167 specia
862 specialist
223 specialists
89 specialized
96 specializes
382 specific
84 specificall
134 specification
290 specifications
130 speed
94 spher
126 sphinx
107 spi
93 spir
623 spok
369 sport
431 sports
1609 spring
121 springfield
93 sprint
104 spss
12712 sql
149 sqlit
101 srm
92 ssas
196 ssh
126 ssis
114 ssl
106 ssrs
284 st
80 sta
108 stabilit
193 stabl
1267 stack
387 staff
88 staffing
108 stag
81 stages
414 stakeholders
89 stand
393 stan

486 водн
163 водоотведен
207 водоочистк
373 водоподготовк
512 водоснабжен
596 воен
329 военнообяза
1177 вожден
2631 возврат
91 возвратн
131 возвраща
84 возвращен
264 возглав
128 возглавля
334 воздейств
140 возделыван
483 воздух
305 воздушн
320 возл
177 возложен
471 возмещен
23710 возмож
138708 возможн
94 вознагражда
6130 вознагражден
2107 возника
804 возникновен
263 возникш
2784 возражен
839 возраст
84 возраста
154 возьм
133 возьмет
328 войковск
160 войт
593 вокза
465 вокруг
82 вол
137 волг
554 волгоград
568 волгоградск
544 волейбол
172 волейбольн
463 волжск
286 волн
139 вологд
108 вологодск
175 волод
165 волоколамск
147 волокон
132 волонтерск
387 волос
302 волс
271 вообщ
214 воплот
364 воплоща
280 воплощен
15770 вопрос
806 воронеж
369 воронежск
518 воронк
375 ворот
118 ворошилов
103 воск
353 воскр
5368 воскресен
80 воскресенск
147 воспользова
305 воспринима
211 восприят
80 воспроизведен
96 воспроизводств
179 восстан
193 восстановительн
1162 восстановлен
668 восток
858 восточн
1283 вос

215 инфографик
159 инфоповод
103 информатизац
281 информатик
21076 информац
16481 информацион
186 информир
343 информирова
3089 информирован
4494 инфраструктур
460 инфраструктурн
1700 инцидент
246 ион
373 ип
402 ипотек
377 ипотечн
508 иркутск
1500 ис
1006 иска
112 исключ
82 исключа
201 исключен
699 исключительн
111 ископа
831 искрен
410 искусств
416 искусствен
459 исп
204 испан
251 испанск
7198 исполнен
1024 исполнител
9198 исполнительн
96 исполнительск
290 исполня
1371 использ
2420 использова
13259 использован
1995 использу
1296 используем
1280 исправлен
123 исправля
1110 исправн
5269 испытан
92 испытател
18794 испытательн
115 испытыва
103 исследова
4705 исследован
88 исследовател
918 исследовательск
190 истечен
118 истин
2006 истор
275 историческ
1927 источник
210 истр
105 истринск
731 исход
1093 исходн
1840 исходя
8778 ит
242 итак
508 итал
772 итальянск
82 итд
80 итерац
7498 итог
477 итогов
106 итр
132 ишимба
12408 ищ
5224 ищет
385 ищеш
159 ищут
100 июл
178 июн
2040 йн
340 йог
133 й

114 обезжелезиван
1409 обеспеч
37183 обеспечен
266 обеспечив
7876 обеспечива
358 обеща
106 обещан
1132 обзвон
372 обзор
1567 обл
4025 облада
242 обладан
176 обладател
295 обладнан
192 облак
32982 област
106 областн
1431 облачн
2432 обм
547 обмен
112 обменива
452 обнаружен
471 обнинск
3507 обновлен
314 обновля
351 обо
93 обобща
130 обобщен
159 обогатительн
221 обогащен
92 обожа
149 обозначен
440 оборачиваем
392 оборон
1210 оборот
133 оборотн
2561 оборудова
48931 оборудован
471 обоснова
828 обоснован
164 обосновыва
361 обособлен
88 обоюдн
1347 обрабатыва
126 обработа
14238 обработк
784 обработок
2631 образ
164 образова
72331 образован
1704 образовательн
2049 образц
989 обрат
2557 обратн
828 обраща
3225 обращен
965 обследован
696 обслуговуван
1800 обслужива
51162 обслуживан
598 обстановк
214 обстоятельств
512 обсуд
13184 обсужда
95 обсуждаем
1158 обсужден
3613 обув
471 обувн
80 обусловлен
196 обустроен
210 обустройств
133 обухов
1167 обуч
9692 обуча
9123 обучаем
107604 обучен
326 обход
56

3110 проспект
82 просрочек
269 просрочен
4495 прост
333 просто
137 простор
1575 просторн
1652 пространств
81 пространствен
275 просчет
2031 просьб
120 прот
84 противодейств
713 противопожарн
90 противопоказан
3750 протокол
596 прототип
366 прототипирован
732 протяжен
903 проф
568 профес
2149 професс
1511 профессиона
16119 профессионал
1252 профессионализм
120733 профессиональн
2284 профил
269 профилактик
938 профилактическ
203 профилирован
7218 профильн
150 профсоюз
256 профсоюзн
2374 проход
86 проходим
276 проходя
4192 прохожден
93 процветан
4989 процедур
4124 процент
132 процентн
202 процес
42746 процесс
218 процессинг
153 процессингов
123 процессн
130 процессор
3381 проч
397 прочита
191 прочл
334 прочн
254 прочност
130 прочностн
167 прош
348 прошедш
515 прошл
1021 прощ
813 прояв
1856 проявлен
534 проявля
143 пруд
7245 прям
200 пс
597 психолог
335 психологическ
142 псковск
7024 пт
771 птиц
85 птицеводств
155 птицефабрик
149 птн
250 пто
205 пу
1438 публикац
597 публичн
288 пуга
239 пу

275 твор
4657 творческ
617 творчеств
352 тг
1533 тд
874 те
309 театр
157 театральн
5864 теб
101 тег
5141 текст
515 текстил
416 текстильн
262 текстильщик
822 текстов
112 текучк
7851 текущ
1025 тел
597 телевиден
141 телевизион
231 телевизор
227 тележк
1019 телек
98 телекана
80 телеканал
162 телеком
1033 телекоммуникац
2268 телекоммуникацион
493 телемаркетинг
326 телемаркетолог
121 телемеханик
21934 телефон
1266 тем
1089 тематик
1073 тематическ
952 темп
167 температур
190 температурн
364 тенг
2177 тенденц
3497 тендер
1621 тендерн
742 теннис
220 теннисн
1084 теор
817 теоретическ
2606 тепл
99 теплиц
124 тепличн
863 теплов
101 теплоизоляц
138 теплообмен
89 теплообменник
176 теплоснабжен
337 теплотехник
174 теплотехническ
186 теплоэнергетик
148 терапевт
99 терешков
320 територ
233 терм
190 термин
405 термина
1242 терминал
271 терминальн
205 терминолог
204 термическ
149 термообработк
167 терпелив
186 терпен
189 террас
29602 территор
3708 территориальн
653 тесн
4984 тест
124 тестер
108 тестир
3

## Шаг 3. Решам задачу через модель Random Forest

In [19]:
#Делаем трейн тест сплит

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

X_train, X_test, y_train, y_test = train_test_split(train_data_features, train_df['target'], test_size=0.33, random_state=42)
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train  (134000, 10000)
X_test  (66000, 10000)
y_train  (134000,)
y_test  (66000,)


In [20]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Пока делаем 100 деревьев, но в следующих вариантах домашки попробую сделать 50
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

# В версии с 5 000 фич обучение заняло 12 минут 41 секунду, старт в 23-25

forest = forest.fit( X_train, y_train )
print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 57min 54s, sys: 1min 57s, total: 59min 52s
Wall time: 6min 34s


In [21]:
predict = forest.predict(X_test)

In [22]:
# прошлый результаты были 0.96 0.96 0.96

print('#'*20,'>>> Random Trees <<<','#'*20,'\n',classification_report(y_test, predict),'\n')

#################### >>> Random Trees <<< #################### 
              precision    recall  f1-score   support

          0       0.97      0.96      0.97     34984
          1       0.96      0.97      0.96     31016

avg / total       0.97      0.97      0.97     66000
 



# Шаг 4. Готовим файл к отправке

In [23]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Делаем random forest на 100 деревьев
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

# В версии с 5 000 фич обучение заняло 22 минут 12 секунд, старт в 23-37
forest = forest.fit( train_data_features, train_df['target'] )

print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 1h 46min 32s, sys: 3min 43s, total: 1h 50min 15s
Wall time: 11min 14s


In [24]:
test_df['concat'] = test_df['name']+' '+ test_df['description']

test_df.shape

(170179, 4)

In [25]:
num_test_vacs = len(test_df['concat'])

In [26]:
%%time
clean_test_vacs = []
print ('Очищаем и собираем нормализованные описания из test_df...\n')
for i in range(0,num_test_vacs):
    if( (i+1) % 10000 == 0 ):
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_test_vacs, (i+1)/num_test_vacs )) 
    clean_test_vacs.append( vac_to_words( test_df['concat'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из test_df...

Описание 10000 из 170179 в обработке... 6%
Описание 20000 из 170179 в обработке... 12%
Описание 30000 из 170179 в обработке... 18%
Описание 40000 из 170179 в обработке... 24%
Описание 50000 из 170179 в обработке... 29%
Описание 60000 из 170179 в обработке... 35%
Описание 70000 из 170179 в обработке... 41%
Описание 80000 из 170179 в обработке... 47%
Описание 90000 из 170179 в обработке... 53%
Описание 100000 из 170179 в обработке... 59%
Описание 110000 из 170179 в обработке... 65%
Описание 120000 из 170179 в обработке... 71%
Описание 130000 из 170179 в обработке... 76%
Описание 140000 из 170179 в обработке... 82%
Описание 150000 из 170179 в обработке... 88%
Описание 160000 из 170179 в обработке... 94%
Описание 170000 из 170179 в обработке... 100%
Я закончила!!!
CPU times: user 22min 23s, sys: 8.08 s, total: 22min 31s
Wall time: 22min 34s


In [27]:
%%time
# Фигачим bag of words по тестовым данным

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()

CPU times: user 16.6 s, sys: 4.72 s, total: 21.3 s
Wall time: 21.5 s


In [28]:
%%time
# Через Random Forest делаем предсказание
result_test = forest.predict(test_data_features)

CPU times: user 45.6 s, sys: 12.6 s, total: 58.2 s
Wall time: 26.1 s


In [29]:
# Копируем результат в датафрейм с полями id и target 
# предыдущий реузльтат был таким http://take.ms/cVRK4
output = pd.DataFrame( data={'id':test_df['id'], 'target':result_test} )
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0


In [30]:
output.to_csv( "Random_Forest_10000stem-and-names.csv", sep=',', encoding='utf-8',index=False )